# Data analysis 2

## Adding some feature engineering

In [3]:
#Setting up environment
import pandas as pd
import os
import numpy as np
from IPython.display import Math

from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
import matplotlib.pylab as plt
import seaborn as sns
from sklearn import metrics, svm,tree,preprocessing
from pprint import pprint
from sklearn.metrics import (auc, confusion_matrix, roc_curve, accuracy_score, precision_score)

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
#Uploading data
df_processed= pd.read_csv("/Suzi fun files/QB course/QB_DS_FinalProject/data/processed/data_all_features.csv", index_col=[0])
df_processed.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis_category
id,,,,,,,,,,,,,,,,,,,,,
842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


## Correlation matrix in Data exploration has shown that there is a lot of multilinearity in the data set. 
## Calculating the variance inflation factor 
https://stackoverflow.com/questions/42658379/variance-inflation-factor-in-python

In [41]:
def variance_inflation_factors(exog_df):
    '''
    Parameters
    ----------
    exog_df : dataframe, (nobs, k_vars)
        design matrix with all explanatory variables, as for example used in
        regression.

    Returns
    -------
    vif : Series
        variance inflation factors
    '''
    exog_df = add_constant(exog_df)
    vifs = pd.Series(
        [1 / (1. - OLS(exog_df[col].values, 
                       exog_df.loc[:, exog_df.columns != col].values).fit().rsquared) 
         for col in exog_df],
        index=exog_df.columns,
        name='VIF'
    )
    return vifs

variance_inflation_factors(df_processed)

const                      1945.673866
radius_mean                3817.259795
texture_mean                 11.891280
perimeter_mean             3792.697001
area_mean                   348.115385
smoothness_mean               8.194309
compactness_mean             51.445960
concavity_mean               71.002747
concave points_mean          60.172431
symmetry_mean                 4.220806
fractal_dimension_mean       15.756978
radius_se                    75.737325
texture_se                    4.205686
perimeter_se                 70.398925
area_se                      41.196467
smoothness_se                 4.070801
compactness_se               15.366350
concavity_se                 15.914022
concave points_se            11.601253
symmetry_se                   5.179151
fractal_dimension_se          9.724753
radius_worst                815.945630
texture_worst                18.606605
perimeter_worst             405.150023
area_worst                  343.494355
smoothness_worst         

## radius_mean has very high VIF, so does perimeter_mean

In [46]:
c = df_processed.corr().abs()
s = c.unstack()
sorted_correlation = s.sort_values(kind="quicksort")
sorted_correlation[900:-31]

area_se          radius_se          0.951830
radius_se        area_se            0.951830
perimeter_worst  area_mean          0.959120
area_mean        perimeter_worst    0.959120
                 area_worst         0.959213
area_worst       area_mean          0.959213
area_mean        radius_worst       0.962746
radius_worst     area_mean          0.962746
perimeter_worst  radius_mean        0.965137
radius_mean      perimeter_worst    0.965137
perimeter_mean   radius_worst       0.969476
radius_worst     perimeter_mean     0.969476
radius_mean      radius_worst       0.969539
radius_worst     radius_mean        0.969539
perimeter_worst  perimeter_mean     0.970387
perimeter_mean   perimeter_worst    0.970387
radius_se        perimeter_se       0.972794
perimeter_se     radius_se          0.972794
perimeter_worst  area_worst         0.977578
area_worst       perimeter_worst    0.977578
radius_worst     area_worst         0.984015
area_worst       radius_worst       0.984015
area_mean 

In [93]:
df_removing_features = df_processed.drop(['radius_mean','area_mean','perimeter_worst','area_worst',
                                          'perimeter_se','concavity_mean','compactness_mean','perimeter_mean',
                                          'area_se','texture_mean','concave points_mean','concavity_worst','compactness_worst',
                                         'concave points_worst','fractal_dimension_mean',
                                          'smoothness_worst','fractal_dimension_worst','radius_se','compactness_se'],axis = 1)
variance_inflation_factors(df_removing_features)

const                   131.117724
smoothness_mean           1.878561
symmetry_mean             3.051059
texture_se                2.838671
smoothness_se             1.945312
concavity_se              3.627044
concave points_se         3.729724
symmetry_se               2.699415
fractal_dimension_se      2.607203
radius_worst              3.125632
texture_worst             2.539236
symmetry_worst            4.362513
diagnosis_category        3.546553
Name: VIF, dtype: float64

In [91]:
c = df_removing_features.corr().abs()
s = c.unstack()
sorted_correlation = s.sort_values(kind="quicksort")
sorted_correlation[150:-14]

compactness_se        symmetry_mean           0.421659
symmetry_mean         compactness_se          0.421659
fractal_dimension_se  smoothness_se           0.427374
smoothness_se         fractal_dimension_se    0.427374
symmetry_mean         symmetry_se             0.449137
symmetry_se           symmetry_mean           0.449137
texture_worst         diagnosis_category      0.456903
diagnosis_category    texture_worst           0.456903
radius_se             concave points_se       0.513346
concave points_se     radius_se               0.513346
smoothness_mean       symmetry_mean           0.557775
symmetry_mean         smoothness_mean         0.557775
diagnosis_category    radius_se               0.567134
radius_se             diagnosis_category      0.567134
concave points_se     fractal_dimension_se    0.611044
fractal_dimension_se  concave points_se       0.611044
symmetry_mean         symmetry_worst          0.699826
symmetry_worst        symmetry_mean           0.699826
radius_se 

In [94]:
df_removing_features.head()

,smoothness_mean,symmetry_mean,texture_se,smoothness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,symmetry_worst,diagnosis_category
id,,,,,,,,,,,,
842302,0.11840,0.2419,0.9053,0.006399,0.05373,0.01587,0.03003,0.006193,25.38,17.33,0.4601,1
842517,0.08474,0.1812,0.7339,0.005225,0.01860,0.01340,0.01389,0.003532,24.99,23.41,0.2750,1
84300903,0.10960,0.2069,0.7869,0.006150,0.03832,0.02058,0.02250,0.004571,23.57,25.53,0.3613,1
84348301,0.14250,0.2597,1.1560,0.009110,0.05661,0.01867,0.05963,0.009208,14.91,26.50,0.6638,1
84358402,0.10030,0.1809,0.7813,0.011490,0.05688,0.01885,0.01756,0.005115,22.54,16.67,0.2364,1


In [96]:
df_removing_features.to_csv("/Suzi fun files/QB course/QB_DS_FinalProject/data/processed/data_cleaned_features.csv")